In [1]:
from crewai import Crew, Agent, Task
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from pydantic import BaseModel
from typing import List

"A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title."

class SubSection(BaseModel):
    title: str
    content: str

class BlogPost(BaseModel):
    title: str
    introduction: str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]


In [3]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content from result of the Senior Researcher.
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

writing_task = Task(
    description="Write a blog post from the research result.",
    agent=editor,
    expected_output=
    "A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
    output_pydantic=BlogPost,
)

research_task = Task(
    description="Research the web for information about {topic}",
    agent=researcher,
    expected_output="A list of sources that are relevant to the topic.",
    output_file="research_result.md",
)

crew = Crew(
    agents=[researcher, editor],
    tasks=[research_task, writing_task],
    verbose=True,
    planning=True,
)

result = crew.kickoff(inputs={
    "topic": "The Deep Learning Research Trends in 2025",
}, )

print(result)

 
[2025-01-11 02:10:00][INFO]: Planning the crew execution
# Agent: Senior Researcher
## Task: Research the web for information about The Deep Learning Research Trends in 20251. The Senior Researcher will start by defining a clear search query relevant to 'Deep Learning Research Trends in 2025'.
2. Utilize the 'Search the internet' tool to input the search query. The tool will search Google to provide the top 10 results which will likely contain relevant articles, research papers, and blog posts.
3. Analyze the search results and select at least 5-7 URLs that appear credible and contain valuable information on the topic.
4. Use the 'Read website content' tool for each of the selected URLs. Input the website URL into the tool to extract and read the content from these relevant sources.
5. From the content gathered, identify key trends, advancements, and major themes discussed regarding deep learning research expected in 2025. Ensure to note these trends and their sources for reference.


In [10]:
print(result.pydantic.sections[0].content)

In 2025, self-supervised learning is expected to become a dominant methodology in deep learning. Unlike traditional supervised learning that relies heavily on labeled datasets, self-supervised learning generates labels from the data itself. This shift significantly reduces the need for manual data labeling, speeding up the training process and enabling AI to learn from vast amounts of unlabeled data. According to experts from [AI and Deep Learning Trends Report](https://www.example-ai-report.com), this approach will enhance the efficiency of models across various domains, including natural language processing and image recognition.
